In [1]:
'''
STEP1: GATHER THE DATA
- Use Selenium/BeautifulSoup to webscrape CARDNAME|CARDTEXT|FACTION(S), then store in csv or xls.
OR
- Find somebody's already made excel with card and text details.
STEP2: REMOVE STOPWORDS AND PERFORM K-MEANS ALG. ON CARDTEXT
- Initially I wanted to group the factions, THEN cluster on the text, but for the first run we will cluster on just the text.
'''


import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline
import re
import requests
from bs4 import BeautifulSoup
# import json
import warnings
import mechanicalsoup
warnings.filterwarnings(action='once')

In [ ]:
"""
ITERATE THROUGH EACH PAGE AND GRAB Each card, go to its page, grab its flavor text + faction(s).
(All this really does is save you time by not having to go into ->"Deck Builder" and
add ALL your cards to a deck and exporting it.... :))
"""
df = pd.DataFrame(columns=['name', 'faction', 'rarity', 'text', 'shiftstone_cost', 'decks_on_site'])
page_num = 1
check_last_pg = re.compile(r'\b(No cards found in your collection)\b')
final_collection = []
browser = mechanicalsoup.Browser(soup_config={'features': 'lxml'})

while True:
    # Now that were logged in pull each link for our collection and grab the info
    current_page = 'https://eternalwarcry.com/cards?p=' + str(page_num)
    page_num = page_num + 1 # Iterate don't wanna forget ;)
    collection_page = browser.get(current_page) # Response object with .soup object
    
    # Get out of loop if we reach 'no cards exists' page.
    if bool(check_last_pg.search(collection_page.text)):
        break
    
    # Current Page. #collection_page.text because its response object
    html_page = collection_page.soup 
    # Get all cards on this page.
    divs = html_page.findAll(class_= 'card-search-item col-lg-3 col-sm-4 col-xs-6 add-card-main element-relative')  
    for div in divs:
        # This is where it gets fun. 
        # Each div is a card from the search_view. a href has the link which contains info.
        card_name = str(div.find_all('a'))
        #print(card_name) # This is to be searched for the name of card; can also acquire set and card#
        card_details = str(div.find(class_ = 'display-count'))

        # STEP TWO: FORMULATE CARD
        # First part of string is count
        count = str(re.search(r'data-count="(\d)"', card_details, re.IGNORECASE).group(1))

        # Now get set - card#
        card_set = str(re.search(r'data-card="(\d+-\d+)"', card_details, re.IGNORECASE).group(1))
        card_set = card_set.split('-')
        details = '(Set' + str(card_set[0]) + ' #' + str(card_set[1]) + ')'

        # Now get card name with grammar
        name = str(re.search(r'<img alt="(.+)" class=', card_name, re.IGNORECASE).group(1))

        # Put it altogether....
        item = count + ' ' + name + ' ' + details
        #print(item)

        # Drop each page into a list for later...
        final_collection.append(item)
        
        
# TODO: ADD SHIFSTONE AMOUNT TO MY_COLLECTION FOR BETTER ANALYSIS 

In [ ]:
# Check how many total cards we have (all 4 of each specific card).
total_cards = 0
for x in final_collection:
    total_cards = total_cards + int(x[0])
total_cards

output_json = {'total_cards': total_cards, 'my_collection': final_collection}

In [ ]:
# Output all cards to JSON or csv + excel.
# with open('my_collection.json', 'w') as fp:
#     json.dump(output_json, fp)
full_all_card = pd.to_excel('all_cards.xlsx')